## 1

In [154]:
import pandas as pd
import numpy as np

data_conversion = lambda a: 1 if a == 'Yes' else 0
vdata_conversion = np.vectorize(data_conversion)
data = pd.read_csv("Carseats.csv") # shape(400,11)
sales = np.array(data["Sales"]) # float64
price = np.array(data["Price"]) # int64 (400,)
urban = vdata_conversion(np.array(data["Urban"])) # (400,)
us = vdata_conversion(np.array(data["US"])) # (400,)
columns_names = ["Price", "Urban", "US", "Sales"]
train_data = pd.DataFrame(np.column_stack((price, urban, us, sales)), columns=columns_names)

### 1-(a)

In [138]:
import statsmodels.formula.api as sm

model_a = sm.ols('Sales ~ Price + Urban + US', data = train_data).fit()

# R^2 scores
print("R^2:", model_a.rsquared)

R^2: 0.23927539218405547


### 1-(b)

x1: Price, x2: Urban, x3: US, bi: 대응되는 xi의 coefficient (i=1,2,3), b0: intercept  
Linear Regression Model(y=b0+b1x1+b2x2+b3x3)에서,  
b1는 x1(Price)를 제외한 다른 predictors들이 고정되어 있을 때, x1(Price)가 1 단위 증가할 때 y(Sales) 값에 주는 평균적인 변화량(average effect)을 의미한다.  
b2는 x2(Urban)를 제외한 다른 predictors들이 고정되어 있을 때, x2(Urban)이 'No'일 때에 대해 'Yes'로 바뀔 때 y(Sales) 값에 주는 평균적인 변화량을 의미한다.  
b3는 x3(US)를 제외한 다른 predictors들이 고정되어 있을 때, x3(US)이 'No'일 때에 대해 'Yes'로 바뀔 때 y(Sales) 값에 주는 평균적인 변화량을 의미한다.

### 1-(c)

In [144]:
# intercept and coefficients
[print("b%i: %s" %(i,j)) for i,j in enumerate(model_a.params)]
print()

b0: 13.043468936764889
b1: -0.05445884917758212
b2: -0.021916150814141434
b3: 1.2005726977941158



y = 13.043468936764889 + -0.05445884917758212x1 + -0.021916150814141434x2 + 1.2005726977941156x3  
(x2, x3 값은 각각 0일 때 'No', 1일 때 'Yes'를 의미함)

### 1-(d)

In [145]:
print(model_a.summary())

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.234
Method:                 Least Squares   F-statistic:                     41.52
Date:                Wed, 06 Apr 2022   Prob (F-statistic):           2.39e-23
Time:                        16:28:00   Log-Likelihood:                -927.66
No. Observations:                 400   AIC:                             1863.
Df Residuals:                     396   BIC:                             1879.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     13.0435      0.651     20.036      0.0

constant와 variables의 p-value(P>|t|)를 살펴 보면, Intercept, Price(x1), US(x3)의 p-value는 매우 작기 때문에(<0.001) null hypothesis를 reject하고, Urban(x2)의 p-value는 매우 크므로(1에 근접) null hypothesis를 reject하지 못한다. 즉 문제에서 j=0,1,3일 때 null hypothesis를 reject한다.

### 1-(e)

In [147]:
model_e = sm.ols('Sales ~ Price + US', data=train_data).fit()
print(model_e.summary())

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.235
Method:                 Least Squares   F-statistic:                     62.43
Date:                Wed, 06 Apr 2022   Prob (F-statistic):           2.66e-24
Time:                        16:29:38   Log-Likelihood:                -927.66
No. Observations:                 400   AIC:                             1861.
Df Residuals:                     397   BIC:                             1873.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     13.0308      0.631     20.652      0.0

null hypothesis를 reject하지 못하는 Urban(x2)를 제외하고, 신뢰수준 0.05에서 통계적으로 유의한 Price(x1), US(x3) 변수만 이용하여 regression model을 얻었다.

### 1-(f)

In [148]:
print("model (a)")
print("R^2 score:", model_a.rsquared)
print("AIC:", model_a.aic)
print("BIC:", model_a.bic)
print()

print("model (e)")
print("R^2 score:", model_e.rsquared)
print("AIC:", model_e.aic)
print("BIC:", model_e.bic)

model (a)
R^2 score: 0.23927539218405547
AIC: 1863.3120738106172
BIC: 1879.277931999049

model (e)
R^2 score: 0.23926288842678567
AIC: 1861.3186484129808
BIC: 1873.2930420543048


model (e)가 model (a)보다 data fitting을 더 잘 하였다. AIC, BIC 값이 낮을 수록 좋은 model이다. (a)~(d)까지 사용한 model보다 (e)에서 사용한 model이 AIC, BIC 값이 더 낮으므로 Part (e)의 model이 data fitting을 더 잘한다. 두 model이 동일한 data set로 fit되었으므로, TSS 값은 같고 model의 predictor 수를 늘릴 수록 RSS 값은 같거나 크다. 따라서 model의 predictor 수를 늘릴 수록 R^2=1-RSS/TSS 값은 같거나 크다. 따라서 R^2 score가 높다고 해서 model이 더 data fitting을 잘 한다는 보장이 없다. 반면 AIC, BIC 값은 RSS 값(혹은 RSS와 동치인 -2log(likelihood)(각각을 maximizing하는 beta가 같다는 관점에서 동치))과 predictor 개수를 둘 다 고려하여 서로 다른 model size를 갖는 model을 비교하는데 더욱 적합한 기준이 된다.

### 1-(g)

In [151]:
print("95% confidence intervals for the coefficients")
print(model_e.conf_int(alpha=.05))

95% confidence intervals for the coefficients
                  0          1
Intercept  11.79032  14.271265
Price      -0.06476  -0.044195
US          0.69152   1.707766


b1의 95% 신뢰구간: [-0.06476, -0.044195]  
b3의 95% 신뢰구간: [0.69152, 1.707766]  

## 2

In [152]:
data2 = pd.read_csv("Default.csv")

balance = np.array(data2["balance"])
income = np.array(data2["income"])
default = vdata_conversion(np.array(data2["default"]))
student = vdata_conversion(np.array(data2["student"]))

x_train2 = np.column_stack((income, balance))

### 2-(a)

In [153]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Split the sample set into a training set and a validation set
X_train, X_val, Y_train, Y_val = train_test_split(x_train2, default, test_size = 0.5, shuffle=True, random_state = 2022)

# Fit a multiple logistic regression model
mlgr = LogisticRegression()
mlgr.fit(X_train, Y_train)

# Equivalent to mlgr.predict(X_val)
# params = np.concatenate((mlgr.intercept_, mlgr.coef_[0]))
log_odds = mlgr.intercept_ + X_val.dot(mlgr.coef_[0])
posterior_prob = np.exp(log_odds) / (1 + np.exp(log_odds))
posterior_prob_classified = np.array([1 if i > 0.5 else 0 for i in posterior_prob])
val_error = sum([0 if i==j else 1 for (i,j) in zip(Y_val, posterior_prob_classified)]) / Y_val.shape[0]

print("validation set error(estimated test error):", end=" ")
print(val_error)

validation set error(estimated test error): 0.0338


### 2-(b)

Logistic Regression Model (p(y=1|x)=exp(b0+b1x1+b2x2)/(1+exp(b0+b1x1+b2x2)))에서  
b1는 balance(x2) predictor가 고정되어 있을 때, income(x1)가 1 단위 증가할 때 Default의 log odds 값에 주는 변화량을 의미한다.  
b2는 income(x1) predictor가 고정되어 있을 때, balance(x2)가 1 단위 증가할 때 Default의 log odds 값에 주는 변화량을 의미한다.  

### 2-(c)

In [155]:
import random as rd

def k_fold_cv(k, X_train, Y_train):
    indices = np.array(range(len(X_train)))
    rd.shuffle(indices)
    k_indices = np.split(indices,k)

    k_x_groups = [X_train[k_indices[i]] for i in range(k)]
    k_y_groups = [Y_train[k_indices[i]] for i in range(k)]
    val_errors = []
    for i in range(k):
        x_val = k_x_groups[i]
        y_val = k_y_groups[i]
        x_train = np.concatenate(k_x_groups[:i] + k_x_groups[i+1:])
        y_train = np.concatenate(k_y_groups[:i] + k_y_groups[i+1:])
        
        mlgr = LogisticRegression()
        mlgr.fit(x_train, y_train)

        log_odds = mlgr.intercept_ + x_val.dot(mlgr.coef_[0])
        posterior_prob = np.exp(log_odds) / (1 + np.exp(log_odds))
        posterior_prob_classified = np.array([1 if i > 0.5 else 0 for i in posterior_prob])
        val_errors.append(sum([0 if i==j else 1 for (i,j) in zip(y_val, posterior_prob_classified)]) / y_val.shape[0])
    return val_errors

rd.seed(2022)
k = 5
val_errors = k_fold_cv(k, x_train2, default)
print(val_errors)
val_error = np.mean(val_errors)
print("5-fold cross-validation error(estimated test error):", end=" ")
print(round(val_error,5))

[0.0235, 0.0215, 0.0325, 0.03, 0.027]
5-fold cross-validation error(estimated test error): 0.0269


2-(a)에서 구한 validation set error는 주어진 train set의 1/2을 이용해 학습한 model로부터 얻은 것이고, 2-(c)에서 구한 k-fold cross-validation error는 주어진 train set의 (k-1)/k을 이용해 학습한 model들의 평균으로 얻은 것이다.  
따라서 bias-variance trade off를 고려하면 test error를 estimate할 때 (c)에서 학습한 model이 (a)에서 학습한 model보다 bias가 낮고 variance가 높다는 것을 알 수 있다.  
실제로 계산된 error 계산 값이 (a) model보다 (c) model이 더 낮은 것으로부터, (a) model이 test error를 overestimate(high bias)하였음을 추론할 수 있다.

### 2-(d)

In [11]:
x_train3 = np.column_stack((balance, income, student))

rd.seed(2022)
k = 5

val_errors = k_fold_cv(k, x_train3, default)

print(val_errors)
val_error = np.mean(val_errors)
print("5-fold cross-validation error(estimated test error) with a dummy variable:", end=" ")
print(round(val_error,5))

[0.025, 0.028, 0.0315, 0.0355, 0.0305]
5-fold cross-validation error(estimated test error) with a dummy variable: 0.0301


dummy variable(student)를 추가하였더니 5-fold cross-validation error(estimated test error)가 증가하였다. 따라서 dummy variable을 없앤 원래 모델 (c)이 test error가 더 낮을 것으로 예상된다.